<a href="https://colab.research.google.com/github/Austfi/SNOWPACKforPatrollers/blob/dev/SNOWPACKforPatrollers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SNOWPACK Model Setup and Execution

This notebook guides you through the process of setting up and running the SNOWPACK model. It includes steps for installing necessary libraries, compiling the SNOWPACK and MeteoIO code, configuring the model, fetching meteorological data from historical weather models, and running SNOWPACK.

# The following cells below set up SNOWPACK AND MeteoIO and the PATH structure to run it. They should not be edited. Pressing the play button below will run them all.

In [1]:
# @title Install environment updates
!apt-get update
!apt-get install -y build-essential cmake git liblapack-dev numdiff

%pip install openmeteo-requests requests-cache retry-requests pandas numpy

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
from pathlib import Path
import openmeteo_requests
import requests_cache
from retry_requests import retry
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import json
from typing import Optional



Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cli.github.com/packages stable InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,398 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,288 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:14 http://se

In [2]:
# @title Download SNOWPACK
!git clone https://github.com/snowpack-model/snowpack.git


Cloning into 'snowpack'...
remote: Enumerating objects: 15270, done.
remote: Counting objects: 100% (989/989), done.
remote: Compressing objects: 100% (329/329), done.
remote: Total 15270 (delta 745), reused 733 (delta 660), pack-reused 14281 (from 2)
Receiving objects: 100% (15270/15270), 84.87 MiB | 10.56 MiB/s, done.
Resolving deltas: 100% (10662/10662), done.
Updating files: 100% (1756/1756), done.


In [3]:
# @title Make  ~/usr folder and remove sample data
!mkdir ~/usr/
!rm -rf /content/sample_data

In [4]:
# @title Compile MeteoIO  (Runs about 6 Mins)
!cd /content/snowpack/Source/meteoio/ && \
cmake -S . -B build-meteoio -DCMAKE_BUILD_TYPE=Release -DCMAKE_INSTALL_PREFIX=$HOME/usr && \
cmake --build build-meteoio -j2 && \
cmake --install build-meteoio

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Performing Test HAVE_STRUCT_STAT_ST_FLAGS
-- Performing Test HAVE_STRUCT_STAT_ST_FLAGS - Failed
-- Performing Test HAVE_FEENABLE
-- Performing Test HAVE_FEENABLE - Success
-- Could NOT find Doxygen (missing: DOXYGEN_EXECUTABLE) 
-- Could NOT find Doxygen (missing: DOXYGEN_EXECUTABLE) 
-- Configuring done (1.2s)
-- Generating done (0.1s)
-- Build files have been written to: /content/snowpack/Source/meteoio/build-meteoio
[  1%] Building C object meteoio/CMakeFiles/meteoio.dir/thirdParty/ct

In [5]:
# @title Compile SNOWPACK (Runs about 2 mins)

!cd /content/snowpack/Source/snowpack/ && \
cmake -S . -B build-snowpack -DCMAKE_BUILD_TYPE=Release \
      -DCMAKE_PREFIX_PATH=$HOME/usr \
      -DDEBUG_ARITHM=OFF \
      -DENABLE_LAPACK=ON \
      -DMETEOIO_LIBRARY=$HOME/usr/lib/libmeteoio.so && \
cmake --build build-snowpack -j2 && \
cmake --install build-snowpack

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found METEOIO 
-- Found Lapack 
-- Performing Test HAVE_FEENABLE
-- Performing Test HAVE_FEENABLE - Success
-- Could NOT find Doxygen (missing: DOXYGEN_EXECUTABLE) 
-- Could NOT find Doxygen (missing: DOXYGEN_EXECUTABLE) 
-- Configuring done (0.5s)
-- Generating done (0.0s)
-- Build files have been written to: /content/snowpack/Source/snowpack/build-snowpack
[  6%] Building CXX object snowpack/CMakeFiles/snowpack.dir/vanGenuchten.cc.o
[  6%] Building CXX object snowpack/CMakeFiles/snowpa

In [ ]:
# @title Set up PATH to compiled code
import os

# Set PATH and LD_LIBRARY_PATH in Python environment
# Note: These paths are for Colab. In local environments, snowpack is typically in /usr/local/bin
os.environ["PATH"] = "/home/app/usr/bin:" + os.environ.get("PATH", "")
os.environ["LD_LIBRARY_PATH"] = "/home/app/usr/lib:" + os.environ.get("LD_LIBRARY_PATH", "")

# Verify snowpack is accessible
!which snowpack

/usr/local/bin/snowpack


# The above cells have set up SNOWPACK and made it ready to run. The below cells create the files, collect data, and run the snowpack model. This generates snowprofiles with no snow to start the model from.

In [ ]:
#@title SNOWPACK Configuration Files Generation
"""
Generate .sno and .ini configuration files for SNOWPACK simulations
"""

#@markdown ## Station Configuration
station_id = "ksp_model"  #@param {"type":"string"}
station_name = "keystone_alpine"  #@param {"type":"string"}
latitude = 39.56858687967004  #@param {"type":"number"}
longitude = -105.91900397453021  #@param {"type":"number"}
altitude = 3614  #@param {"type":"integer"}
altitude_unit = "meters"  # @param ["meters", "feet"]
timezone = -7  #@param {"type":"number"}
profile_date = "2024-11-01T00:00:00"  #@param {"type":"string"}
coord_sys = "UTM" #@param {"type":"string"}
coord_param = "13S" #@param {"type":"string"}

#@markdown ## Virtual Slopes Configuration
num_slopes = 5  #@param {"type":"number", "min":1, "max":10}
include_flat = True  #@param {"type":"boolean"}
default_slope_angle = 38.0  #@param {"type":"number", "min":0, "max":90}

#@markdown ## Slope Directions (degrees: 0=North, 90=East, 180=South, 270=West)
north_slope = True  #@param {"type":"boolean"}
east_slope = True  #@param {"type":"boolean"}
south_slope = True  #@param {"type":"boolean"}
west_slope = True  #@param {"type":"boolean"}
custom_directions = ""  #@param {"type":"string"} {description:"Comma-separated azimuth angles (e.g., 45,135,225,315)"}

#@markdown ## SNOWPACK Settings
meas_tss = "false"  #@param ["true","false"]
enforce_measured_snow_heights = "false"  #@param ["true","false"]
write_profiles = "true"  #@param ["true","false"]
write_timeseries = "false"  #@param ["true","false"]
write_snowpack = "false"  #@param ["true","false"]

#@markdown ## Output Configuration
sno_directory = "/content/input"  #@param {"type":"string"}
ini_directory = "/content/config"  #@param {"type":"string"}
generate_config_files = True  #@param {"type":"boolean"}

# @markdown ### SNOWPACK Run End Date
snowpack_end_date_input = "2025-04-01"  # @param {type:"date"}
snowpack_end_date = f"{snowpack_end_date_input}T00:00"

# For local testing - use current directory
import os
if not os.path.exists("/content"):
    sno_directory = "."
    ini_directory = "."

def generate_slopes(include_flat, north_slope, east_slope, south_slope, west_slope, custom_directions, default_slope_angle):
    """Generate list of virtual slopes based on user selections"""
    slopes = []

    # Add flat slope if requested
    if include_flat:
        slopes.append((0.0, 0.0))

    # Add cardinal direction slopes
    if north_slope:
        slopes.append((default_slope_angle, 0.0))
    if east_slope:
        slopes.append((default_slope_angle, 90.0))
    if south_slope:
        slopes.append((default_slope_angle, 180.0))
    if west_slope:
        slopes.append((default_slope_angle, 270.0))

    # Add custom directions
    if custom_directions.strip():
        try:
            custom_angles = [float(x.strip()) for x in custom_directions.split(',')]
            for angle in custom_angles:
                if 0 <= angle <= 360:
                    slopes.append((default_slope_angle, angle))
        except ValueError:
            print("Warning: Invalid custom directions format. Use comma-separated numbers.")

    return slopes

def to_meters(value: float, unit: str) -> float:
    """Normalize altitude inputs so downstream code always works in meters."""
    unit = unit.lower()
    if unit == "meters":
        return float(value)
    if unit == "feet":
        return float(value) * 0.3048
    raise ValueError(f"Unsupported altitude unit: {unit}")

# Using variables from the configuration cell
altitude_meters = to_meters(altitude, altitude_unit)

def create_sno_content(station_id, station_name, longitude, latitude, altitude_meters, timezone, profile_date, slope_angle, slope_azimuth):
    """Create .sno file content for a single slope"""

    content = f"""SMET 1.1 ASCII
[HEADER]
station_id       = {station_id}
station_name     = {station_name}
longitude        = {longitude}
latitude         = {latitude}
altitude         = {altitude_meters}
nodata           = -999
tz               = {timezone}
source           = OpenMeteo
prototype        = SNOWPACK
ProfileDate      = {profile_date}
HS_Last          = 0.0000
SlopeAngle       = {slope_angle}
SlopeAzi         = {slope_azimuth}
nSoilLayerData   = 0
nSnowLayerData   = 0
SoilAlbedo       = 0.09
BareSoil_z0      = 0.020
CanopyHeight     = 0.00
CanopyLeafAreaIndex = 0.00
CanopyDirectThroughfall = 1.00
ErosionLevel     = 0
TimeCountDeltaHS = 0.000000
WindScalingFactor = 1.00

fields           = timestamp Layer_Thick  T  Vol_Frac_I  Vol_Frac_W  Vol_Frac_V  Vol_Frac_S Rho_S Conduc_S HeatCapac_S  rg  rb  dd  sp  mk mass_hoar ne CDot metamo
[DATA]
"""

    return content

def create_ini_content(station_id, filename, snowfiles, meas_tss, enforce_measured_snow_heights, 
                        coord_sys, coord_param, timezone, write_profiles, write_timeseries, write_snowpack):
    """Create .ini file content with multiple SNOWFILE entries
    
    Args:
        station_id: Station identifier
        filename: SMET filename
        snowfiles: List of .sno filenames
        meas_tss: MEAS_TSS setting ("true" or "false")
        enforce_measured_snow_heights: ENFORCE_MEASURED_SNOW_HEIGHTS setting ("true" or "false")
        coord_sys: Coordinate system (e.g., "UTM")
        coord_param: Coordinate parameter (e.g., "13S")
        timezone: Timezone offset (e.g., -7)
        write_profiles: PROF_WRITE setting ("true" or "false")
        write_timeseries: TS_WRITE setting ("true" or "false")
        write_snowpack: SNOW_WRITE setting ("true" or "false")
    
    Returns:
        String containing .ini file content
    """

    content = f"""[General]
BUFFER_SIZE = 370
BUFF_BEFORE = 1.5

[Input]
COORDSYS = {coord_sys}
COORDPARAM = {coord_param}
TIME_ZONE = {timezone}

METEO = SMET
METEOPATH = ../input
METEOFILE1 = {filename}
"""

    # Add SNOWFILE entries for each slope
    for i, snowfile in enumerate(snowfiles, 1):
        content += f"SNOWFILE{i} = {snowfile}\n"

    content += f"""
[Output]
COORDSYS = {coord_sys}
COORDPARAM = {coord_param}
TIME_ZONE = {timezone}
METEOPATH = ./output
EXPERIMENT = res
SNOW_WRITE = {write_snowpack.lower()}

TS_WRITE = {write_timeseries.lower()}
TS_FORMAT = SMET
TS_START = 0.0
TS_DAYS_BETWEEN = 0.125
PROF_WRITE = {write_profiles.lower()}
PROF_FORMAT = PRO
AGGREG_PRF = false
PROF_START = 0.0
PROF_DAYS_BETWEEN = 0.125

[Snowpack]
MEAS_TSS = {meas_tss.lower()}
ENFORCE_MEASURED_SNOW_HEIGHTS = {enforce_measured_snow_heights.lower()}
FORCING = ATMOS
SW_MODE = INCOMING
MEAS_INCOMING_LONGWAVE = false
HEIGHT_OF_WIND_VALUE = 4.5
HEIGHT_OF_METEO_VALUES = 4.5
ATMOSPHERIC_STABILITY = MO_MICHLMAYR
ROUGHNESS_LENGTH = 0.002
CALCULATION_STEP_LENGTH = 15.0
CHANGE_BC = false
THRESH_CHANGE_BC = -1.0
SNP_SOIL = false
SOIL_FLUX = false
GEO_HEAT = 0.06
CANOPY = false

[SnowpackAdvanced]
FIXED_POSITIONS = 0.25 0.5 1.0 -0.25 -0.10
SNOW_EROSION = true
WIND_SCALING_FACTOR = 1.0
NUMBER_SLOPES = {len(snowfiles)}
SNOW_REDISTRIBUTION = TRUE

[Filters]
ENABLE_METEO_FILTERS = true
PSUM::filter1 = min
PSUM::arg1::soft = true
PSUM::arg1::min = 0.0
TA::filter1 = min_max
TA::arg1::min = 240
TA::arg1::max = 320
RH::filter1 = min_max
RH::arg1::min = 0.01
RH::arg1::max = 1.2
RH::filter2 = min_max
RH::arg2::soft = true
RH::arg2::min = 0.05
RH::arg2::max = 1.0
ISWR::filter1 = min_max
ISWR::arg1::min = -10
ISWR::arg1::max = 1500
ISWR::filter2 = min_max
ISWR::arg2::soft = true
ISWR::arg2::min = 0
ISWR::arg2::max = 1500
RSWR::filter1 = min_max
RSWR::arg1::min = -10
RSWR::arg1::max = 1500
RSWR::filter2 = min_max
RSWR::arg2::soft = true
RSWR::arg2::min = 0
RSWR::arg2::max = 1500
ILWR::filter1 = min_max
ILWR::arg1::min = 188
ILWR::arg1::max = 600
ILWR::filter2 = min_max
ILWR::arg2::soft = true
ILWR::arg2::min = 200
ILWR::arg2::max = 400
TSS::filter1 = min_max
TSS::arg1::min = 200
TSS::arg1::max = 320
TSG::filter1 = min_max
TSG::arg1::min = 200
TSG::arg1::max = 320
HS::filter1 = min
HS::arg1::soft = true
HS::arg1::min = 0.0
HS::filter2 = rate
HS::arg2::max = 5.55e-5
VW::filter1 = min_max
VW::arg1::min = -2
VW::arg1::max = 70
VW::filter2 = min_max
VW::arg2::soft = true
VW::arg2::min = 0.0
VW::arg2::max = 50.0

[Interpolations1D]
MAX_GAP_SIZE = 86400
PSUM::resample1 = accumulate
PSUM::ARG1::period = 900
HS::resample1 = linear
HS::ARG1::MAX_GAP_SIZE = 43200
VW::resample1 = nearest
VW::ARG1::extrapolate = true
DW::resample1 = nearest
DW::ARG1::extrapolate = true

[Generators]
ILWR::generator1 = AllSky_LW
ILWR::arg1::type = Carmona
ILWR::arg1::shade_from_dem = FALSE
ILWR::arg1::use_rswr = FALSE
ILWR::generator2 = ClearSky_LW
ILWR::arg2::type = Dilley
"""

    return content

def get_slope_filename(angle, azimuth, slope_index, include_flat):
    """Get filename for slope naming convention.
    
    Naming rules:
    - Flat slope (angle == 0.0): Returns 'keystone' (no number)
    - Non-flat slopes: Returns 'keystone1', 'keystone2', etc.
    - Numbering depends on whether flat slope is included:
      * If flat included: non-flat slopes start from index 1
      * If flat not included: all slopes start from index 1
    
    Args:
        angle: Slope angle in degrees (0.0 for flat)
        azimuth: Slope azimuth in degrees
        slope_index: Index of slope in the slopes list
        include_flat: Whether flat slope is included in the list
    
    Returns:
        Filename base (without .sno extension)
    """
    if angle == 0.0:
        return "keystone"  # Flat slope gets no number
    else:
        # For numbered slopes, determine the number based on position
        if include_flat:
            # If flat is included, non-flat slopes start from index 1
            return f"keystone{slope_index}"
        else:
            # If no flat, start numbering from 1
            return f"keystone{slope_index + 1}"

def main():
    """Main execution function"""

    print("SNOWPACK Configuration Files Generation")
    print("=" * 50)
    print(f"Station: {station_name} ({station_id})")
    print(f"Location: {latitude:.6f}°N, {longitude:.6f}°W, {altitude_meters}m")
    print(f"Sno Directory: {sno_directory}")
    print(f"Ini Directory: {ini_directory}")
    print("=" * 50)

    if not generate_config_files:
        print("Configuration generation disabled. Set 'generate_config_files' to True to generate files.")
        return

    # Validate profile_date format
    try:
        from datetime import datetime
        datetime.strptime(profile_date, "%Y-%m-%dT%H:%M:%S")
    except ValueError:
        print(f"Warning: profile_date format may be incorrect. Expected format: YYYY-MM-DDTHH:MM:SS")
        print(f"  Current value: {profile_date}")

    # Validate directories
    try:
        os.makedirs(sno_directory, exist_ok=True)
        os.makedirs(ini_directory, exist_ok=True)
    except Exception as e:
        print(f"Error creating directories: {e}")
        return

    # Generate slopes
    slopes = generate_slopes(include_flat, north_slope, east_slope, south_slope, west_slope, custom_directions, default_slope_angle)

    # Limit to requested number of slopes
    if len(slopes) > num_slopes:
        slopes = slopes[:num_slopes]
        print(f"Warning: Limited to {num_slopes} slopes as requested")

    # Create multiple .sno files and collect snowfile names
    snowfiles = []
    sno_files_created = []

    for i, (angle, azimuth) in enumerate(slopes):
        # Get filename based on new naming convention
        sno_filename = get_slope_filename(angle, azimuth, i, include_flat) + ".sno"
        snowfiles.append(sno_filename)

        # Create .sno content for this slope
        sno_content = create_sno_content(station_id, station_name, longitude, latitude, altitude_meters, timezone, profile_date, angle, azimuth)

        # Write .sno file to input directory
        sno_filepath = os.path.join(sno_directory, sno_filename)
        try:
            with open(sno_filepath, "w") as f:
                f.write(sno_content)
            sno_files_created.append(sno_filepath)
        except Exception as e:
            print(f"Error writing {sno_filepath}: {e}")
            continue

    # Create .ini content with multiple SNOWFILE entries
    ini_content = create_ini_content(
        station_id, 
        f"{station_id}.smet", 
        snowfiles, 
        meas_tss, 
        enforce_measured_snow_heights,
        coord_sys,
        coord_param,
        timezone,
        write_profiles,
        write_timeseries,
        write_snowpack
    )

    # Write .ini file to keystone directory
    ini_filename = os.path.join(ini_directory, f"{station_id}.ini")
    try:
        with open(ini_filename, "w") as f:
            f.write(ini_content)
    except Exception as e:
        print(f"Error writing {ini_filename}: {e}")
        return

    # Display results
    print(f"\n Configuration files created:")
    print(f"   .ini file: {ini_filename}")
    print(f"   .sno files:")
    for sno_file in sno_files_created:
        print(f"     {sno_file}")

    print(f"\nVirtual slopes configured: {len(slopes)}")
    for i, (angle, azimuth) in enumerate(slopes):
        sno_filename = get_slope_filename(angle, azimuth, i, include_flat)
        if angle == 0.0:
            print(f"   Slope {i+1}: Flat (0°) -> {sno_filename}.sno")
        else:
            direction = ""
            if azimuth == 0.0:
                direction = "North"
            elif azimuth == 90.0:
                direction = "East"
            elif azimuth == 180.0:
                direction = "South"
            elif azimuth == 270.0:
                direction = "West"
            else:
                direction = f"{azimuth}°"
            print(f"   Slope {i+1}: {angle}° slope facing {direction} -> {sno_filename}.sno")

    print(f"\nSNOWPACK settings:")
    print(f"   MEAS_TSS = {meas_tss}")
    print(f"   ENFORCE_MEASURED_SNOW_HEIGHTS = {enforce_measured_snow_heights}")
    print(f"   NUMBER_SLOPES = {len(slopes)}")

    print("\n🎉 SNOWPACK configuration generation complete!")

# Run the main function
if __name__ == "__main__":
    main()

SNOWPACK Configuration Files Generation
Station: keystone_alpine (ksp_model)
Location: 39.568587°N, -105.919004°W, 3614.0m
Sno Directory: /content/input
Ini Directory: /content/config

 Configuration files created:
   .ini file: /content/config/ksp_model.ini
   .sno files:
     /content/input/keystone.sno
     /content/input/keystone1.sno
     /content/input/keystone2.sno
     /content/input/keystone3.sno
     /content/input/keystone4.sno

Virtual slopes configured: 5
   Slope 1: Flat (0°) -> keystone.sno
   Slope 2: 38.0° slope facing North -> keystone1.sno
   Slope 3: 38.0° slope facing East -> keystone2.sno
   Slope 4: 38.0° slope facing South -> keystone3.sno
   Slope 5: 38.0° slope facing West -> keystone4.sno

SNOWPACK settings:
   MEAS_TSS = false
   ENFORCE_MEASURED_SNOW_HEIGHTS = false
   NUMBER_SLOPES = 5

🎉 SNOWPACK configuration generation complete!


In [ ]:
# @title SMET Handling, API Setup, and Parameters

# @markdown ### Location Settings are used from above.
# Pulling from the configuration cell
# latitude = 39.56858687967004  # @param {type:"number"}
# longitude = -105.91900397453021  # @param {type:"number"}
# altitude_input = 3614  # @param {type:"number"}
# altitude_unit = "meters"  # @param ["meters", "feet"]

# @markdown ### Station Information is used from above.
# Pulling from the configuration cell
# station_name = "keystone_model"  # @param {type:"string"}

# @markdown ### Time Period
start_date = "2024-11-01"  # @param {type:"date"}
end_date = "2025-04-30"    # @param {type:"date"}

# @markdown ### Weather Model
model_selection = "nbm"  # @param ["nbm", "ifs", "gfs"]

# @markdown ### Open-Meteo Elevation
openmeteo_elevation_mode = "use_selected_altitude"  # @param ["use_model_elevation", "use_selected_altitude"]

# @markdown ### SMET Generation
generate_files = True  # @param {type:"boolean"}

# --- Helper functions ----------------------------------------------------


def resolve_openmeteo_elevation(mode: str, station_alt_m: float) -> Optional[float]:
    """
    Decide which elevation to send to the Open-Meteo API.
    - use_model_elevation     -> None (API uses its DEM; statistical downscaling stays enabled)
    - use_selected_altitude   -> the altitude value converted to meters above
    """
    mode = mode.lower()
    if mode == "use_model_elevation":
        return None
    if mode == "use_selected_altitude":
        return float(station_alt_m)
    raise ValueError(f"Unknown elevation mode: {mode}")


def create_smet_from_weather_data(weather_df: pd.DataFrame,
                                  output_path: str,
                                  station_id: str,
                                  station_name: str,
                                  latitude: float,
                                  longitude: float,
                                  altitude_meters: float,
                                  timezone: float = -7) -> None:
    """
    Create a SMET file from a weather DataFrame and write it to disk.
    
    Args:
        weather_df: DataFrame containing weather data with timestamp column
        output_path: Path where SMET file will be written
        station_id: Station identifier
        station_name: Station name
        latitude: Latitude in degrees
        longitude: Longitude in degrees
        altitude_meters: Altitude in meters
        timezone: Timezone offset (default: -7 for Mountain Time)
    """
    if "timestamp" in weather_df.columns:
        weather_df = weather_df.copy()
        weather_df["timestamp"] = pd.to_datetime(weather_df["timestamp"])
    else:
        weather_df = weather_df.copy()
        weather_df.reset_index(inplace=True)
        weather_df["timestamp"] = pd.to_datetime(weather_df["timestamp"])

    fields = ["timestamp", "TA", "RH", "TSG", "VW", "DW", "ISWR", "PSUM"]
    if "HS" in weather_df.columns:
        fields.append("HS")

    output_dir = os.path.dirname(output_path)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)

    with open(output_path, "w") as f:
        f.write("SMET 1.1 ASCII\n")
        f.write("[HEADER]\n")
        f.write(f"station_id = {station_id}\n")
        f.write(f"station_name = {station_name}\n")
        f.write(f"latitude = {latitude:.10f}\n")
        f.write(f"longitude = {longitude:.10f}\n")
        f.write(f"altitude = {altitude_meters}\n")
        f.write("nodata = -777\n")
        f.write(f"Tz = {timezone}\n")
        f.write(f"fields = {'\t'.join(fields)}\n")

        field_count = len(fields)
        units_offset = ["0"] * field_count
        units_multiplier = ["1"] * field_count
        f.write(f"units_offset     = {' '.join(units_offset)}\n")
        f.write(f"units_multiplier = {' '.join(units_multiplier)}\n")

        f.write("[DATA]\n")
        for _, row in weather_df.iterrows():
            values = []
            for field in fields:
                if field == "timestamp":
                    values.append(row[field].strftime("%Y-%m-%dT%H:%M:%S"))
                else:
                    val = row.get(field, -777)
                    if pd.isna(val):
                        val = -777
                    values.append(f"{val:.2f}")
            f.write("\t".join(values) + "\n")


def fetch_openmeteo_historical(lat: float,
                               lon: float,
                               start_date: str,
                               end_date: str,
                               model: str = "gfs_global",
                               elevation: Optional[float] = None) -> Optional[pd.DataFrame]:
    """
    Fetch historical forecast data from the Open-Meteo API.
    """
    cache_session = requests_cache.CachedSession(".cache", expire_after=3600)
    retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
    openmeteo = openmeteo_requests.Client(session=retry_session)

    url = "https://historical-forecast-api.open-meteo.com/v1/forecast"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start_date,
        "end_date": end_date,
        "hourly": [
            "temperature_2m",
            "relative_humidity_2m",
            "wind_speed_10m",
            "wind_direction_10m",
            "snow_depth",
            "direct_radiation",
            "precipitation",
        ],
        "models": model,
        "wind_speed_unit": "ms",
    }

    if elevation is not None:
        params["elevation"] = elevation

    print(f"Fetching {model} data for {lat}, {lon} from {start_date} to {end_date}")
    if "elevation" in params:
        print(f"Open-Meteo elevation parameter: {params['elevation']} (m)")

    try:
        responses = openmeteo.weather_api(url, params=params)
        response = responses[0]

        print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
        print(f"Elevation: {response.Elevation()} m asl")

        hourly = response.Hourly()
        hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
        hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
        hourly_wind_speed_10m = hourly.Variables(2).ValuesAsNumpy()
        hourly_wind_direction_10m = hourly.Variables(3).ValuesAsNumpy()
        hourly_snow_depth = hourly.Variables(4).ValuesAsNumpy()
        hourly_direct_radiation = hourly.Variables(5).ValuesAsNumpy()
        hourly_precipitation = hourly.Variables(6).ValuesAsNumpy()

        time_index = pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left",
        )

        df = pd.DataFrame(
            {
                "timestamp": time_index,
                "TA": hourly_temperature_2m + 273.15,   # Convert to Kelvin
                "RH": hourly_relative_humidity_2m / 100.0,
                "VW": hourly_wind_speed_10m,
                "DW": hourly_wind_direction_10m,
                "HS": hourly_snow_depth,
                "ISWR": hourly_direct_radiation,
                "PSUM": hourly_precipitation,
            }
        )
        # TSG (ground temperature) is not available from Open-Meteo API
        # Using assumed value of 273.15 K (0°C) as a reasonable default
        # SNOWPACK will calculate actual ground temperature during simulation
        df["TSG"] = 273.15
        df = df.replace([np.inf, -np.inf], np.nan)

        print(f"Fetched {len(df)} records")
        return df

    except Exception as e:
        print(f"Error fetching data: {e}")
        return None


# --- Derived values / summary -------------------------------------------

# Validate dates
try:
    from datetime import datetime
    start_dt = datetime.strptime(start_date, "%Y-%m-%d")
    end_dt = datetime.strptime(end_date, "%Y-%m-%d")
    if start_dt >= end_dt:
        raise ValueError(f"start_date ({start_date}) must be before end_date ({end_date})")
except ValueError as e:
    print(f"Date validation error: {e}")
    print("Please check your date inputs and try again.")
    raise

# Validate that altitude_meters is available (from configuration cell)
try:
    altitude_meters
except NameError:
    print("Error: altitude_meters not found. Please run the configuration cell first.")
    raise

# altitude_meters = altitude # Commented out direct assignment
openmeteo_elevation = resolve_openmeteo_elevation(openmeteo_elevation_mode, altitude_meters)

print("SMET Generation Parameters")
print("=" * 50)
print(f"Location: {latitude}, {longitude}")
print(f"Altitude input: {altitude} {altitude_unit} ({altitude_meters:.2f} m used for calculations)")
# print(f"Altitude: {altitude_meters:.2f} m")
print(f"Station: {station_name}")
print(f"Period: {start_date} to {end_date}")
print(f"Model: {model_selection}")
print(f"Generate: {generate_files}")
if openmeteo_elevation is None:
    print("Open-Meteo elevation: model elevation (automatic downscaling)")
else:
    print(f"Open-Meteo elevation: {openmeteo_elevation:.2f} m (selected altitude)")
print("=" * 50)

SMET Generation Parameters
Location: 39.56858687967004, -105.91900397453021
Altitude input: 3614 meters (3614.00 m used for calculations)
Station: keystone_alpine
Period: 2024-11-01 to 2025-04-30
Model: nbm
Generate: True
Open-Meteo elevation: 3614.00 m (selected altitude)


In [ ]:
# @title Generate SMET Files
# @markdown Run this cell to generate SMET files with the parameters above

# Hardcoded output path
# output_directory = "/content/input" # Commented out hardcoded path

# Use sno_directory from the configuration cell
output_directory = sno_directory  # sno directory is input folder

# Model mapping
model_mapping = {
    "nbm": "ncep_nbm_conus",
    "ifs": "ecmwf_ifs",
    "gfs": "gfs_global"
}


if generate_files:
    print("Starting SMET generation...")

    # Parse model selection
    selected_models = [model_selection]

    # Validate and map models
    valid_models = ['nbm', 'ifs', 'gfs']
    selected_models = [model for model in selected_models if model in valid_models]

    if not selected_models:
        print("No valid models selected!")
        print(f"Available models: {', '.join(valid_models)}")
    else:
        print(f"Processing {len(selected_models)} model(s): {', '.join(selected_models)}")
        print("="*50)

        successful_files = []

        for model in selected_models:
            print(f"\nProcessing {model}...")

            # Map to Open-Meteo model name
            openmeteo_model = model_mapping[model]

            # Fetch data
            df = fetch_openmeteo_historical(
                lat=latitude,
                lon=longitude,
                start_date=start_date,
                end_date=end_date,
                model=openmeteo_model,
                elevation=openmeteo_elevation
            )

            if df is None or df.empty:
                print(f"Failed to fetch data for {model}")
                continue

            # Create output filename with simplified naming
            output_file = os.path.join(output_directory, f"{station_id}.smet")
            # could add in _{model} if wanted to specify

            try:
                # Generate SMET file
                create_smet_from_weather_data(
                    weather_df=df,
                    output_path=output_file,
                    station_id=f"{station_id}_{model}",
                    station_name=f"{station_name}_{model}",
                    latitude=latitude,
                    longitude=longitude,
                    altitude_meters=altitude_meters,
                    timezone=timezone
                )

                print(f"SMET file created: {output_file}")
                print(f"Records: {len(df)}")
                print(f"Period: {df['timestamp'].min()} to {df['timestamp'].max()}")

                successful_files.append(output_file)

            except Exception as e:
                print(f"Error creating SMET for {model}: {e}")
                continue

        print(f"\nSuccessfully generated {len(successful_files)} SMET files:")
        for file in successful_files:
            print(f"  {file}")

        if successful_files:
            print(f"\nFiles saved to: {output_directory}")
else:
    print("Generation disabled. Set 'generate_files' to True to generate SMET files.")

Starting SMET generation...
Processing 1 model(s): nbm

Processing nbm...
Fetching ncep_nbm_conus data for 39.56858687967004, -105.91900397453021 from 2024-11-01 to 2025-04-30
Open-Meteo elevation parameter: 3614.0 (m)
Coordinates: 39.561767578125°N -105.92127990722656°E
Elevation: 3614.0 m asl
Fetched 4344 records
SMET file created: /content/input/ksp_model.smet
Records: 4344
Period: 2024-11-01 00:00:00+00:00 to 2025-04-30 23:00:00+00:00

Successfully generated 1 SMET files:
  /content/input/ksp_model.smet

Files saved to: /content/input


In [ ]:
# @title Run this cell to run SNOWPACK
import subprocess
import sys

# Construct the path to the .ini file using variables from the configuration cell
ini_filepath = os.path.join(ini_directory, f"{station_id}.ini")

# Find the snowpack executable path dynamically
try:
    result = subprocess.run(["which", "snowpack"], capture_output=True, text=True, check=True)
    snowpack_executable_path = result.stdout.strip()
except (subprocess.CalledProcessError, FileNotFoundError):
    snowpack_executable_path = "snowpack"  # Fallback to PATH lookup

# Verify .ini file exists
if not os.path.exists(ini_filepath):
    raise FileNotFoundError(f"Configuration file not found: {ini_filepath}")

# Run SNOWPACK with proper error handling and real-time output streaming
# In Jupyter notebooks, we need to ensure output streams properly
try:
    # Use subprocess.run without capturing output to stream to notebook
    # stdout and stderr set to None means they go to the notebook's output
    result = subprocess.run(
        [snowpack_executable_path, "-c", ini_filepath, "-e", snowpack_end_date],
        cwd=ini_directory,
        check=True,
        stdout=None,  # Stream stdout to notebook
        stderr=None   # Stream stderr to notebook
    )
except subprocess.CalledProcessError as e:
    print(f"\nError running SNOWPACK (exit code {e.returncode})")
    print("Check the output above for error messages.")
    raise
except FileNotFoundError:
    print(f"SNOWPACK executable not found: {snowpack_executable_path}")
    print("Please ensure SNOWPACK is compiled and in PATH")
    raise

[i] [2025-10-30T15:18:10] ---> Start SNOWPACK in RESEARCH mode
[i] []                 /usr/local/bin/snowpack compiled on Oct 30 2025 at 15:14:43
[i] []                 Experiment : res
[i] []                 Output dir : ./output

[i] []                 Run on meteo station ksp_model_nbm
[i] []                 Reading snow cover data for station ksp_model_nbm
[i] []                 Finished initializing station ksp_model_nbm
[i] []                 No file ./output/ksp_model_nbm_res-4.ini to erase
[i] [2025-10-30T15:18:10] ---> Start simulation for ksp_model_nbm on 2024-10-31T23:45:00-07:00
[i] []                 End date specified by user: 2025-04-01T00:00:00-07:00
[i] []                 Integration step length: 15.000000 min
[i] [2024-11-16T00:00:00] ---> Station ksp_model_nbm (5 slope(s)): advanced to 16.11.2024 00:00:00 station time
[i] [2024-12-01T00:00:00] ---> Station ksp_model_nbm (5 slope(s)): advanced to 01.12.2024 00:00:00 station time
[i] [2024-12-16T00:00:00] ---> Station 

In [ ]:
# @title Download .pro files and Open niViz
# @markdown # 5 files will be download.
# @markdown # 1 = flat, 2=N, 3=E, 4=S, 5=W
import glob, shutil, os
from IPython.display import FileLink, Markdown

# Check if running in Colab
try:
    from google.colab import files
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# Determine temp directory based on environment
if IN_COLAB:
    temp_dir = "/content"
else:
    # For local environments, use current directory or a temp directory
    temp_dir = os.getcwd()

# Locate all .pro files
pro_files = glob.glob(os.path.join(ini_directory, 'output', '*.pro'))

if pro_files:
    # Copy all files to a simple location
    downloaded_files = []
    for pro_file in pro_files:
        filename = os.path.basename(pro_file)
        temp_file_path = os.path.join(temp_dir, filename)
        shutil.copy(pro_file, temp_file_path)
        downloaded_files.append(filename)

    # Create a zip file with all .pro files
    import zipfile
    zip_filename = 'snowpack_profiles.zip'
    zip_path = os.path.join(temp_dir, zip_filename)
    with zipfile.ZipFile(zip_path, 'w') as zipf:
        for filename in downloaded_files:
            file_path = os.path.join(temp_dir, filename)
            zipf.write(file_path, filename)

    # Download the zip file (only in Colab)
    if IN_COLAB:
        files.download(zip_path)
    else:
        print(f"Zip file created at: {zip_path}")
        print(f"Files included: {', '.join(downloaded_files)}")

    # Show how to open in niViz
    display(Markdown(f"""
    ---
    {len(downloaded_files)} SNOWPACK profile files have been downloaded as **`{zip_filename}`**.

    Next Step — View in niViz
    1. Go to https://run.niviz.org
    2. Click "File" → "Open Profile" or drag any of the downloaded files into the screen
    3. Select any of the downloaded files:
    """))

    for filename in downloaded_files:
        display(Markdown(f"   - **`{filename}`**"))

else:
    display(Markdown("No .pro files found in the output directory."))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


    ---
    5 SNOWPACK profile files have been downloaded as **`snowpack_profiles.zip`**.

    Next Step — View in niViz
    1. Go to https://run.niviz.org
    2. Click "File" → "Open Profile" or drag any of the downloaded files into the screen
    3. Select any of the downloaded files:
    

   - **`ksp_model_nbm1_res.pro`**

   - **`ksp_model_nbm2_res.pro`**

   - **`ksp_model_nbm3_res.pro`**

   - **`ksp_model_nbm_res.pro`**

   - **`ksp_model_nbm4_res.pro`**

# Optional: Random Forest Instability Analysis

This section uses a Random Forest model (Mayer et al., The Cryosphere, 2022) to calculate probability of instability (P_unstable) from your SNOWPACK .pro files. This provides additional avalanche risk assessment beyond the standard SNOWPACK output.

**Note**: This section is optional and runs independently from the SNOWPACK simulation above. You can analyze any .pro file, including the ones generated above.

**Reference**: [Mayer et al. (2022) - A random forest model to assess snow instability from simulated snow stratigraphy](https://tc.copernicus.org/articles/16/4593/2022/tc-16-4593-2022.pdf)


In [ ]:
# @title Install Python dependencies for RF model
# @markdown Installs scikit-learn and other dependencies needed for the Random Forest instability model

!pip -q install --upgrade pip
!pip -q install numpy pandas matplotlib joblib scikit-learn

import sys
import platform
import sklearn

print("Python:", sys.version)
print("scikit-learn:", sklearn.__version__)
print("✓ Dependencies installed")


In [ ]:
# @title Download RF model and helper scripts
# @markdown Downloads the Random Forest model and helper modules from WSL/SLF repository

import os
import subprocess

# Create directory for RF model files
rf_dir = "/content/rf_instability" if os.path.exists("/content") else "./rf_instability"
os.makedirs(rf_dir, exist_ok=True)

print("Downloading RF model files from WSL/SLF repository...")
print("=" * 60)

# Try to clone the repository (multiple endpoints for resilience)
repo_cloned = False
if not os.path.exists(os.path.join(rf_dir, "..", "random_forest_snow_instability_model")):
    endpoints = [
        "https://git.wsl.ch/mayers/random_forest_snow_instability_model.git",
        "https://code.wsl.ch/mayers/random_forest_snow_instability_model.git",
        "https://gitlabext.wsl.ch/mayers/random_forest_snow_instability_model.git"
    ]
    
    for endpoint in endpoints:
        try:
            result = subprocess.run(
                ["git", "clone", "--depth", "1", endpoint],
                cwd=os.path.dirname(rf_dir),
                capture_output=True,
                timeout=30
            )
            if result.returncode == 0:
                repo_cloned = True
                print(f"✓ Cloned repository from {endpoint}")
                break
        except Exception as e:
            continue

# Copy files if repository was cloned
if repo_cloned:
    repo_path = os.path.join(os.path.dirname(rf_dir), "random_forest_snow_instability_model")
    files_to_copy = [
        ("scripts/get_RF.py", "get_RF.py"),
        ("scripts/readProfile.py", "readProfile.py"),
        ("scripts/plt_RF.py", "plt_RF.py"),
        ("RF_instability_model.sav", "RF_instability_model.sav")
    ]
    
    for src, dst in files_to_copy:
        src_path = os.path.join(repo_path, src)
        dst_path = os.path.join(rf_dir, dst)
        if os.path.exists(src_path):
            os.makedirs(os.path.dirname(dst_path) if os.path.dirname(dst_path) else rf_dir, exist_ok=True)
            import shutil
            shutil.copy(src_path, dst_path)
            print(f"✓ Copied {dst}")

# Download any missing files directly via raw URLs
missing_files = []
files_to_fetch = {
    "get_RF.py": "https://code.wsl.ch/mayers/random_forest_snow_instability_model/-/raw/main/scripts/get_RF.py",
    "readProfile.py": "https://code.wsl.ch/mayers/random_forest_snow_instability_model/-/raw/main/scripts/readProfile.py",
    "plt_RF.py": "https://code.wsl.ch/mayers/random_forest_snow_instability_model/-/raw/main/scripts/plt_RF.py",
    "RF_instability_model.sav": "https://code.wsl.ch/mayers/random_forest_snow_instability_model/-/raw/main/RF_instability_model.sav?inline=false"
}

import urllib.request

for filename, url in files_to_fetch.items():
    file_path = os.path.join(rf_dir, filename)
    if not os.path.exists(file_path):
        try:
            print(f"Downloading {filename}...")
            urllib.request.urlretrieve(url, file_path)
            print(f"✓ Downloaded {filename}")
        except Exception as e:
            missing_files.append(filename)
            print(f"⚠ Failed to download {filename}: {e}")

if missing_files:
    print(f"\n⚠ Warning: Could not download: {', '.join(missing_files)}")
    print("Some features may not work.")
else:
    print("\n✓ All RF model files downloaded successfully")
    print(f"Files in {rf_dir}:")
    for f in os.listdir(rf_dir):
        if os.path.isfile(os.path.join(rf_dir, f)):
            print(f"  - {f}")


In [ ]:
# @title Import modules and load the RF model
# @markdown Loads the Random Forest model and helper functions

import sys
import pathlib
import joblib

# Determine RF directory based on environment
if os.path.exists("/content"):
    rf_dir = pathlib.Path("/content/rf_instability").resolve()
else:
    rf_dir = pathlib.Path("./rf_instability").resolve()

if str(rf_dir) not in sys.path:
    sys.path.insert(0, str(rf_dir))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

try:
    import get_RF
    import readProfile
    import plt_RF
    print("✓ Helper modules imported")
except ImportError as e:
    print(f"⚠ Error importing helper modules: {e}")
    print("Make sure the RF model download cell ran successfully.")
    raise

MODEL_PATH = rf_dir / 'RF_instability_model.sav'
feature_names = ['viscdefrate','rcflat','sphericity','grainsize','penetrationdepth','slab_rhogs']

try:
    model = joblib.load(MODEL_PATH)
    print(f"✓ Loaded RF model: {MODEL_PATH.name}")
except Exception as e:
    print("❗ Model load failed.")
    print("\nThis model was trained with scikit-learn 0.22.1 (Python 3.7.x).")
    print("If unpickling fails under newer scikit-learn, two options:")
    print("  (1) Run locally in a small conda env with scikit-learn==0.22.1;")
    print("  (2) Re-export the estimator to a version-agnostic format (e.g., skops) in a compatible env.")
    print(f"\nOriginal error:\n{e}")
    raise


In [ ]:
# @title Single Profile: Calculate P_unstable
# @markdown Analyze a single SNOWPACK profile for probability of instability

# @markdown ## Profile Selection
use_generated_profiles = True  # @param {type:"boolean"}
# @markdown If False, specify a custom .pro file path below
custom_pro_file = ""  # @param {type:"string"}

# @markdown ## Analysis Parameters
slope_angle = 35  # @param {type:"number", min:0, max:90}
# @markdown Date/time to analyze (format: YYYY-MM-DD HH:MM)
analysis_date = "2025-01-15 11:00"  # @param {type:"string"}

import datetime
import glob

# Determine which .pro file to use
if use_generated_profiles:
    # Find .pro files from SNOWPACK output
    pro_files = glob.glob(os.path.join(ini_directory, 'output', '*.pro'))
    if pro_files:
        # Use the first .pro file (flat slope)
        pro_file = pro_files[0]
        print(f"Using generated profile: {os.path.basename(pro_file)}")
    else:
        print("⚠ No .pro files found in output directory.")
        print("Please run SNOWPACK first or specify a custom file.")
        raise FileNotFoundError("No .pro files found")
else:
    if not custom_pro_file or not os.path.exists(custom_pro_file):
        print("⚠ Custom .pro file not found or not specified.")
        raise FileNotFoundError(f"File not found: {custom_pro_file}")
    pro_file = custom_pro_file
    print(f"Using custom profile: {pro_file}")

# Parse date
try:
    timestamp = datetime.datetime.strptime(analysis_date, "%Y-%m-%d %H:%M")
except ValueError:
    print(f"⚠ Date format error. Using default: 2025-01-15 11:00")
    timestamp = datetime.datetime(2025, 1, 15, 11, 0)

# Read profile and calculate P_unstable
try:
    prof = readProfile.read_profile(pro_file, timestamp, remove_soil=True)
    df_prof = get_RF.create_RFprof(prof, slope_angle, model)
    
    # Quick sanity check: probabilities within [0,1]
    assert df_prof['P_unstable'].between(0, 1).all(), "P_unstable values must be between 0 and 1"
    
    print(f"✓ Profile loaded and analyzed")
    print(f"  Date: {timestamp}")
    print(f"  Slope angle: {slope_angle}°")
    print(f"  Max P_unstable: {df_prof['P_unstable'].max():.3f}")
    print(f"  Depth at max P_unstable: {df_prof.loc[df_prof['P_unstable'].idxmax(), 'layer_top']:.2f} m")
    
    # Plot
    fig, ax = plt.subplots(figsize=(5, 6))
    plt_RF.plot_sp_single_P0(fig, ax, df_prof, var='P_unstable', colorbar=True)
    plt.title(f"P_unstable Analysis\n{os.path.basename(pro_file)} - {timestamp.strftime('%Y-%m-%d %H:%M')}")
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    print(f"Error analyzing profile: {e}")
    print(f"\nTroubleshooting:")
    print(f"  - Check that the date '{timestamp}' exists in the .pro file")
    print(f"  - Verify the .pro file format is correct")
    print(f"  - Try a different date from the file")
    raise


In [ ]:
# @title Time Series: Daily Evolution of P_unstable
# @markdown Analyze how P_unstable evolves over time for a seasonal period

# @markdown ## Profile Selection
use_generated_profiles_ts = True  # @param {type:"boolean"}
custom_pro_file_ts = ""  # @param {type:"string"}

# @markdown ## Time Period
year = 2025  # @param {type:"integer"}
start_month = 12  # @param {type:"integer", min:1, max:12}
start_day = 1  # @param {type:"integer", min:1, max:31}
end_month = 4  # @param {type:"integer", min:1, max:12}
end_day = 1  # @param {type:"integer", min:1, max:31}

# @markdown ## Analysis Parameters
slope_angle_ts = 35  # @param {type:"number", min:0, max:90}

import datetime
import glob

# Determine which .pro file to use
if use_generated_profiles_ts:
    pro_files = glob.glob(os.path.join(ini_directory, 'output', '*.pro'))
    if pro_files:
        pro_file = pro_files[0]
        print(f"Using generated profile: {os.path.basename(pro_file)}")
    else:
        print("⚠ No .pro files found in output directory.")
        raise FileNotFoundError("No .pro files found")
else:
    if not custom_pro_file_ts or not os.path.exists(custom_pro_file_ts):
        raise FileNotFoundError(f"File not found: {custom_pro_file_ts}")
    pro_file = custom_pro_file_ts
    print(f"Using custom profile: {pro_file}")

# Create date range
start = datetime.datetime(year-1 if start_month == 12 else year, start_month, start_day, 12, 0)
stop = datetime.datetime(year, end_month, end_day, 12, 0)

print(f"Analyzing time series from {start.date()} to {stop.date()}")
print(f"Slope angle: {slope_angle_ts}°")

# Read all profiles from file
profiles = readProfile.read_profile(pro_file, remove_soil=True)
dates = pd.date_range(start, stop, freq='D')

df_list = []
missing_dates = []

for ts in dates:
    if ts in profiles['data'].keys():
        prof = profiles['data'][ts]
        if (len(prof.keys()) == 0) or (len(prof['height']) == 0):
            # Empty profile - create placeholder
            df0 = pd.DataFrame(columns=['P_unstable','layer_top','density','hardness','graintype',
                                        'viscdefrate','rcflat','sphericity','grainsize',
                                        'penetrationdepth','slab_rhogs','HS'], index=[0])
            df0['HS'] = 0.0
        else:
            df0 = get_RF.create_RFprof(prof, slope_angle_ts, model)
            df0['HS'] = df0['layer_top'].iloc[-1]
        df0.insert(0, 'datetime', ts)
        df_list.append(df0)
    else:
        missing_dates.append(ts)

if missing_dates:
    print(f"⚠ Warning: {len(missing_dates)} dates not found in profile file")

if not df_list:
    print("⚠ No data found for the specified date range")
    print("Try adjusting the date range or check the .pro file contents")
else:
    df_evo = pd.concat(df_list, ignore_index=True)
    
    print(f"✓ Analyzed {len(df_list)} profiles")
    
    # Plot
    fig, ax = plt.subplots(figsize=(10, 6))
    plt_RF.plot_evo_SP(df_evo, fig, ax, start, stop, var='P_unstable', colorbar=True, resolution='D')
    plt.title(f"Daily Evolution of P_unstable\n{os.path.basename(pro_file)} - Slope: {slope_angle_ts}°")
    plt.tight_layout()
    plt.show()


In [ ]:
# @title Export Daily Summary CSV
# @markdown Generate a CSV file with daily summary statistics for easy analysis

# @markdown ## Export Parameters
export_year = 2025  # @param {type:"integer"}
export_start_month = 12  # @param {type:"integer", min:1, max:12}
export_start_day = 1  # @param {type:"integer", min:1, max:31}
export_end_month = 4  # @param {type:"integer", min:1, max:12}
export_end_day = 1  # @param {type:"integer", min:1, max:31}
export_slope_angle = 35  # @param {type:"number", min:0, max:90}

use_generated_for_export = True  # @param {type:"boolean"}
custom_pro_file_export = ""  # @param {type:"string"}

import datetime
import glob

# Determine which .pro file to use
if use_generated_for_export:
    pro_files = glob.glob(os.path.join(ini_directory, 'output', '*.pro'))
    if pro_files:
        pro_file = pro_files[0]
    else:
        raise FileNotFoundError("No .pro files found")
else:
    if not custom_pro_file_export or not os.path.exists(custom_pro_file_export):
        raise FileNotFoundError(f"File not found: {custom_pro_file_export}")
    pro_file = custom_pro_file_export

start = pd.Timestamp(export_year-1 if export_start_month == 12 else export_year, export_start_month, export_start_day, 12, 0)
stop = pd.Timestamp(export_year, export_end_month, export_end_day, 12, 0)

print(f"Generating daily summary from {start.date()} to {stop.date()}")
print(f"Slope angle: {export_slope_angle}°")

profiles = readProfile.read_profile(pro_file, remove_soil=True)
rows = []

for ts in pd.date_range(start, stop, freq='D'):
    prof = profiles['data'].get(ts)
    if not prof or len(prof.get('height', [])) == 0:
        continue
    
    try:
        dfi = get_RF.create_RFprof(prof, export_slope_angle, model)
        rows.append({
            'datetime': ts,
            'HS': float(dfi['layer_top'].iloc[-1]),
            'P_unstable_max': float(dfi['P_unstable'].max()),
            'z_Pmax': float(dfi.loc[dfi['P_unstable'].idxmax(), 'layer_top']),
            'P_unstable_mean': float(dfi['P_unstable'].mean())
        })
    except Exception as e:
        print(f"⚠ Error processing {ts}: {e}")
        continue

if rows:
    out = pd.DataFrame(rows).sort_values('datetime')
    
    # Determine output path
    if os.path.exists("/content"):
        out_path = '/content/p_unstable_daily.csv'
    else:
        out_path = './p_unstable_daily.csv'
    
    out.to_csv(out_path, index=False)
    
    print(f"\n✓ Daily summary exported to: {out_path}")
    print(f"  Records: {len(out)}")
    print(f"\nPreview:")
    print(out.head(10).to_string(index=False))
    
    # Download in Colab
    try:
        from google.colab import files
        files.download(out_path)
        print("\n✓ File downloaded")
    except ImportError:
        print(f"\nFile saved at: {out_path}")
else:
    print("⚠ No data found for the specified date range")


# This cell can erase input and ini files to start over.  

In [ ]:
# @title Clean up generated files
# @markdown Run this cell to remove generated .sno and .ini files.

run_cleanup = False #@param {type:"boolean"}

import os
import shutil

# Use variables from the configuration cell
sno_directory_to_clear = sno_directory
ini_directory_to_clear = ini_directory

def clear_directory(directory_path):
    """Removes all files and subdirectories within a given directory.
    
    Args:
        directory_path: Path to directory to clear
        
    Returns:
        tuple: (success: bool, files_deleted: int, errors: list)
    """
    if not os.path.exists(directory_path):
        print(f"Directory not found: {directory_path}")
        return False, 0, []
    
    if not os.path.isdir(directory_path):
        print(f"Error: {directory_path} is not a directory")
        return False, 0, []
    
    # Count files before deletion
    file_count = len([f for f in os.listdir(directory_path) 
                      if os.path.isfile(os.path.join(directory_path, f))])
    
    print(f"Clearing contents of: {directory_path}")
    print(f"  Found {file_count} file(s) and {len([d for d in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, d))])} subdirectory(ies)")
    
    files_deleted = 0
    errors = []
    
    for item in os.listdir(directory_path):
        item_path = os.path.join(directory_path, item)
        try:
            if os.path.isfile(item_path) or os.path.islink(item_path):
                os.unlink(item_path)
                files_deleted += 1
            elif os.path.isdir(item_path):
                shutil.rmtree(item_path)
                files_deleted += 1
        except Exception as e:
            error_msg = f"Failed to delete {item_path}: {e}"
            errors.append(error_msg)
            print(f"  Warning: {error_msg}")
    
    if errors:
        print(f"  Completed with {len(errors)} error(s)")
    else:
        print(f"  Successfully deleted {files_deleted} item(s)")
    
    return len(errors) == 0, files_deleted, errors

if run_cleanup:
    print("=" * 60)
    print("WARNING: This will delete all files in the following directories:")
    print(f"  - {sno_directory_to_clear}")
    print(f"  - {ini_directory_to_clear}")
    print("=" * 60)
    
    # Clear the input (sno) directory
    sno_success, sno_count, sno_errors = clear_directory(sno_directory_to_clear)
    
    # Clear the config (ini) directory
    ini_success, ini_count, ini_errors = clear_directory(ini_directory_to_clear)
    
    total_errors = len(sno_errors) + len(ini_errors)
    if total_errors == 0:
        print("\n✓ Cleanup complete successfully.")
    else:
        print(f"\n⚠ Cleanup completed with {total_errors} error(s).")
else:
    print("Cleanup is disabled. Check the 'Run cleanup' box to enable.")

Clearing contents of: /content/input
Clearing contents of: /content/config

Cleanup complete.
